<span style="color:pink"> Seguimiento de Experimentos con MLflow</span>


In [10]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


Como habíamos mencionado en la intro del tracking experiment tiene muchas ventajas y es una de las mejores prácticas que podemos implementar en nuestro flujo de trabajo. En este notebook vamos a ver como podemos hacer tracking de nuestros experimentos con mlflow con diferentes modos de uso, es bueno que tengas en cuenta las diferentes opciones que te ofrece mlflow para hacer tracking de tus experimentos. Si esto te interesa tanto como a mí: 

<div style="text-align: center;">
  <img src="https://pbs.twimg.com/media/FURj-YsXoAAoXGo.jpg" alt="Descripción de la imagen" width="200" />
</div>


### Escenario 1: mlflow en localhost



<div style="text-align: center;">
  <img src="https://mlflow.org/docs/latest/_images/scenario_1.png" alt="Descripción de la imagen" width="300" />
</div>

Esta es la forma más común de ejecutar MLflow, en este caso, MLflow se ejecuta en el mismo host donde se ejecuta el código de Python. Para ejecutar MLflow en localhost, simplemente ejecuta el siguiente comando en la línea de comandos:

```bash
mlflow ui


Lo anterior iniciará un servidor web de MLflow en el puerto 5000 (por defecto). Para acceder a la interfaz de usuario de MLflow, abre http://localhost:5000 en tu navegador. Si presentas problemas con el puerto puedes especificar otro puerto ejecutando el comando 
```bash
mlflow ui --port
```

También podremos crear una carpeta de forma local e indicarle a mlflow que lo fijamos como *_set_tracking_uri_*
En la celda a continuación te enseñaré cómo hacerlo:

In [11]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/jam/Mlops-platzi/tracking/mlruns'


In [12]:
mlflow.set_experiment("iris_experiment")  #nombre delmlflow

with mlflow.start_run(run_name = "example_1"):

    X,y = load_iris(return_X_y= True)  #cargamos features y datos
    params = {"C": 0.1, "random_state": 42}  #cargamos hiperparámetros para el modelo regre logís
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)  #entrenamos nuestro modelo
    y_pred = lr.predict(X)  #hacemos predicción
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))  #calculamos precisión accuracy y guardamos en mlflow
    mlflow.sklearn.log_model(lr, "model")  #guardamos artifact com model
    print(f"default artifacts location: '{mlflow.get_artifact_uri()}'")  #obtenemos uri de interfaz mlflow
    

2024/08/18 06:46:45 INFO mlflow.tracking.fluent: Experiment with name 'iris_experiment' does not exist. Creating a new experiment.


default artifacts location: 'file:///home/jam/Mlops-platzi/tracking/mlruns/298745449306604066/b27fffbfb8c34d3b9c4b64affa2b6292/artifacts'


/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Folder diferente a mlruns, se puede? Si, puedes crear una carpeta y especificarle a mlflow que lo fijamos como *_set_tracking_uri_*
. Para abrir el tracking folder en la termina, debemos especificar el path de la carpeta que creamos y ejecutar el siguiente comando:

mlflow ui --backend-store-uri file:////ruta_de_acceso_a_la_carpeta




In [13]:
mlflow.end_run()  #aseguramos que no esté corriendo otro proceso

In [15]:
mlflow.set_experiment("iris_experiment_ml_local")
mlflow.set_tracking_uri("/home/jam/Mlops-platzi/tracking/experiment_ml")

with mlflow.start_run(run_name = "example_1"):

    X,y = load_iris(return_X_y= True)  #cargamos features y datos
    params = {"C": 0.1, "random_state": 42}  #cargamos hiperparámetros para el modelo regre logís
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)  #entrenamos nuestro modelo
    y_pred = lr.predict(X)  #hacemos predicción
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))  #calculamos precisión accuracy y guardamos en mlflow
    mlflow.sklearn.log_model(lr, "model")  #guardamos artifact com model
    print(f"default artifacts location: '{mlflow.get_artifact_uri()}'")  #obtenemos uri de interfaz mlflow

# en la terminal vamos a estar a nivel de la carpeta donde se encuentra el archivo con los runs de mlflow   
# mlflow ui --backend-store-uri file://///home/jam/Mlops-platzi/tracking/experiment_ml

2024/08/18 07:00:19 INFO mlflow.tracking.fluent: Experiment with name 'iris_experiment_ml_local' does not exist. Creating a new experiment.


default artifacts location: '/home/jam/Mlops-platzi/tracking/experiment_ml/176614702265954641/ac46cd7f58d843e5a19225c8b23481f5/artifacts'


/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Escenario 2: MLflow en localhost con SQLite

Muchos usuarios también ejecutan MLflow en sus máquinas locales con una base de datos compatible con SQLAlchemy : SQLite . En este caso, los artefactos se almacenan en el ./mlrunsdirectorio local y las entidades de MLflow se insertan en un archivo de base de datos SQLite mlruns.db. Es bastante similar al escenario 1, pero usamos como back a sqlite. En lo personal adora los dos escenarios, sin embargo, también podemos usar un bucket en aws como back. Más adelante lo veremos. 

Para abrir el tracking folder con db como back, ejecutamos lo siguiente:

mlflow ui --backend-store-uri sqlite:///backend.db


<div style="text-align: center;">
  <img src="https://mlflow.org/docs/latest/_images/scenario_2.png" alt="Descripción de la imagen" width="300" />
</div>



In [16]:
mlflow.set_tracking_uri("sqlite:///backend.db")  #utilizaremos un backend asociado a sqlite
mlflow.set_experiment("experiment_backend")  #ponemos un nombre único

with mlflow.start_run(run_name = "example_1"):

    X,y = load_iris(return_X_y= True)  #cargamos features y datos
    params = {"C": 0.1, "random_state": 42}  #cargamos hiperparámetros para el modelo regre logís
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)  #entrenamos nuestro modelo
    y_pred = lr.predict(X)  #hacemos predicción
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))  #calculamos precisión accuracy y guardamos en mlflow
    mlflow.sklearn.log_model(lr, artifact_path="models")  #guardamos artifact com model
    print(f"default artifacts location: '{mlflow.get_artifact_uri()}'")  #obtenemos uri de interfaz mlflow

2024/08/18 07:14:11 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/08/18 07:14:11 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

default artifacts location: '/home/jam/Mlops-platzi/tracking/mlruns/1/be069f067a0942879d99f3f00a0a2673/artifacts'


/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/jam/anaconda3/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Escenario 3: MLflow con servidor de seguimiento remoto, backend y almacenes de artefactos

MLflow también admite arquitecturas distribuidas, donde el servidor de seguimiento, el almacén backend y el almacén de artefactos residen en hosts remotos. Este escenario de ejemplo muestra una arquitectura con un servidor de seguimiento de MLflow remoto, una base de datos de Postgres para el almacenamiento de entidades backend y un depósito de S3 para el almacenamiento de artefactos. Con este escensario se puede compartir el proyecto entre varios dddata scientist o machine learning engineers. 

<div style="text-align: center;">
  <img src="https://mlflow.org/docs/latest/_images/scenario_4.png" alt="Descripción de la imagen" width="400" />
</div>

_____

Setup escenario 1: 

* tracking server: no
* backend: local
* artifacts y metadata: local

Setup escenario 2:

* tracking server: sí, sqlite como local server 
* backend store: sqlite database
* artifacts store: local filessystem

Setup escenario 3:

* tracking server: sí, remote server (EC2)
* backend store: Postgres database
* artifacts store: S3 bucket
